In [1]:
from src.agentic import Agent
from src.config import get_pretrained_config
cfg = get_pretrained_config(repo_id="unsloth/Phi-4-mini-instruct-GGUF",
                            filename="Phi-4-mini-instruct-Q5_K_M.gguf")
cfg.model.use_prompt = False
agent = Agent(cfg)

INFO:src.runner.slm_runner:Loading pretrained model from unsloth/Phi-4-mini-instruct-GGUF


Loading pretrained model from unsloth/Phi-4-mini-instruct-GGUF


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96

In [2]:
system_msg = """
You are a helpful AI assistant with access to tool. Before acting on a step, ALWAYS list out your upcoming actions in less than or equal to 3 parts. When you need to use a tool, list what you're gonna do, with what values and then call:
1. Output a JSON object, with 'name' and 'parameters'. For example:
   {"name": "calculator", "parameters": {"expression": "3.14 * a **2"}}
   {"name": "get_date", "parameters": {}}
   {"name": "python_shell", "parameters": {"code": "<code using standard libraries>"}}
   {"name": "get_weather_details", "parameters": {"location": "<location>"}}
2. Wait for the tool result, once you get it, say something like "Now that we have the `result_value`, we can proceed with the next step."
3. Continue your response using the tool result

Available tools:
- calculator: Evaluates simple mathematical expressions (requires "expression" parameter). The calculator supports the following functions:
  "sin", "cos", "tan", "log", "exp", "sqrt", "floor", "ceil", "asin",
  "acos", "atan", "degrees", "radians", "pi", "e", "pow".
- get_date: Returns current date in YYYYMMDD format (requires empty {} parameter)
- python_shell: Executes Python code and returns its result (requires "code" parameter)
- get_weather_details: Fetches weather details for a location (requires "location" parameter)

Tool results will be provided in the format: [Tool <name> returned: <result> | $result_N], always use $result_N to refer to the result of the Nth tool call. You cannot refer to subvalues within $result_N, always use the whole value.

If you encounter errors, or are asked something which the tools dont support switch to the python_shell tool and always use standard libraries, double-check variable names, return types and print outputs.

Never run the same tool twice with the same values or pass empty parameters except for get_date.

Never stimulate the response or caclulate it manually, always use the tools to get the result, if the tool response doesn't come fix your json call and try again.

Once all steps in the prompt are completed, summarize write an end message and print a tick mark (✓) at the end of your step to indicate completion.

The minute you type "USER" the system will crash, be cautious.
"""

# system_msg = """You are a precise and methodical AI assistant with strict access to predefined tools, if you use "USER" or "ASSISTANT" the system would crash. To avoid hallucinations, always adhere meticulously to these instructions:

# Before performing ANY actions, analyze the users query for any mistakes mention and correct them and clearly enumerate ALL upcoming steps you intend to take. Clearly specify tool usage with exact values beforehand. Once your plan is confirmed CLOSELY STICK TO IT, strictly follow these steps:

# - For tool usage, output a single JSON object strictly formatted, the parameter must never be empty unless stated otherwise. The format is:
#   {"name": "tool_name", "parameters": {"param": "exact_value"}}
  
# Tools available:
# - Calculator:
#   {"name": "calculator", "parameters": {"expression": "3.14 * r**r"}}
# - Date retrieval:
#   {"name": "get_date", "parameters": {}}
# - Python execution:
#   {"name": "python_shell", "parameters": {"code": "<code using ONLY standard libraries>"}}
# - Weather details:
#   {"name": "get_weather_details", "parameters": {"location": "New York"}}

# After each tool call:
# 1. Wait explicitly for the response.
# 2. Upon receiving a result, acknowledge explicitly: "Now that we have `$result_N`, proceed with the next step."
# 3. Integrate `$result_N` directly into your next steps. Do NOT speculate or fabricate intermediate steps or results.

# Tool responses format:
# - [Tool <name> returned: <result> | $result_N]
# - Always refer ONLY to the complete `$result_N`.
# - NEVER attempt to access sub-values of `$result_N`.
# - NEVER repeat tool calls with the same parameters or use empty parameters (except for `get_date`).

# Important constraints:
# - If any requested functionality is unsupported directly by tools, default explicitly to the `python_shell` tool, strictly using standard Python libraries ONLY.
# - Double-check ALL variable names, return types, and print outputs.
# - If a tool response fails or is invalid, fix your JSON call precisely and retry.

# Completion:
# - Upon successful completion of all listed steps, explicitly summarize your results clearly.
# - End your final step with a tick mark (✓) for clarity.
# """

# user = user_q  # Use the user_q directly without formatting

# user = """Please perform the following multi-step reasoning and calculation task, using all available tools as needed:
# 1. Calculate the sum of the squares of the first 10 prime numbers.
# 2. Find the date 100 days from today."""

In [3]:
# system_msg = """You are a precise and methodical AI assistant with access to the following tools:

# 1. calculator — Evaluates BASIC mathematical expressions, no matrices or integrals, use python for that. Takes: {"expression": "<string>"}
#    Supported: sin, cos, tan, log, exp, sqrt, floor, ceil, asin, acos, atan, degrees, radians, pi, e, pow
# 2. get_date — Returns current date in YYYYMMDD format. Takes: {}
# 3. python_shell — Executes Python code using standard libraries only. Takes: {"code": "<string>"}
# 4. get_weather_details — Returns weather data. Takes: {"location": "<string>"}

# Instructions:
# - ALWAYS begin by listing upcoming steps (≤3).
# - For tool calls, respond with a SINGLE JSON like:
#   {"name": "tool_name", "parameters": {"param": "value"}}
# - Tool Output format:
#   [Tool <name> returned: <result> | $result_N], call results using $result_N.
# - Never imitate or fabricate results, always use tools to get the exact output, if no output your json was probably wrong, fix it and try again.
# - Acknowledge tool output using: “Now that we have <output from tool>, proceed to...”
# - On tool failure or unsupported input, switch to python_shell using standard libs only.
# - Never repeat a tool call with the same input or use empty params (except get_date).
# - When all steps are done, summarize and end with a ✓.
# - Never ever perform manual calculations or estimations, always use the tools to get the result.

# At the end of your task, ALWAYS output: USER
# """

user = """Calculate the area of a circle with radius 7.25 in 1 line"""

In [4]:
for tok in agent.chat(system_msg, user):
    print(tok, end="")
#    print()

To calculate the area of a circle with radius 7.25, I will use the formula A = π * r^2 and plug in the value for pi (π) as approximately 3.14159 from standard libraries.

I am going to call:
{"name": "calculator", "parameters": {"expression": "3.14159 * pow(7.25, 2)"}}



Processing tool call: calculator with args {'expression': '3.14159 * pow(7.25, 2)'}
Raw call: {"name": "calculator", "parameters": {"expression": "3.14159 * pow(7.25, 2)"}}
Calculating: 3.14159 * pow(7.25, 2)
[calculator → 165.129824375 | id $result_1] Now that we have the `result_value` of 165.129824375, which represents the area in square units for a circle with radius 7.25 using π approximated as 3.14159.

✓



In [ ]:
3.14 * 7.25 ** 2

In [ ]:
# Qwery 1
"""Please help me with a complex analysis:

1. Calculate the area of a circle with radius 7.25
2. Get today's date and add it to the area (treating date as YYYYMMDD)
3. Take the square root of the result
4. Create a 3x3 matrix where:
   - First row is [1, 2, 3]
   - Second row is [4, 5, 6]
   - Third row is [7, 8, 9]
5. Calculate its determinant
6. Compare if the determinant is greater than the square root from step 3
7. Find the current weather in NYC"""

# Qwery 2

"""Please perform the following multi-step reasoning and calculation task, using all available tools as needed:

1. Calculate the sum of the squares of the first 10 prime numbers.
2. Find the date 100 days from today.
3. Concatenate the result from step 1 and the date from step 2 as a string, then calculate the SHA256 hash of this string using Python.
4. Create a 4x4 matrix where the first row is [2, 3, 5, 7], the second row is [11, 13, 17, 19], the third row is [23, 29, 31, 37], and the fourth row is [41, 43, 47, 53].
5. Calculate the inverse of this matrix (if it exists) and sum all the elements of the inverse matrix.
6. Compare the sum from step 5 to the integer value of the first 8 digits of the hash from step 3 (interpreted as a hexadecimal number).
7. Get the current weather in Tokyo and extract only the temperature and weather description.
8. Using Python, create a summary sentence that includes the results of steps 1, 2, 6, and 7."""

In [ ]:
# system_msg = """You are a dumb AI assistant with access to very smart tools. Before acting on a query, list out all your upcoming actions. When you need to use a tool, list what you're gonna do along with the formula(if any) and values and then call with the following format:
# 1. Output a JSON object with 'name' and 'parameters'. For example:
#    {"name": "calculator", "parameters": {"expression": "25 * 48"}}
# 2. Once you get the tool's response, say something like "Now that we have the `result_value`, we can proceed with the next step."
# 3. Continue your response using the tool result

# Tool results will be provided in the format: [Tool <name> returned: <result>]

# If you encounter errors, or are asked something which the tools dont support switch to the python_shell tool and always use standard libraries, double-check variable names, return types and print outputs.

# ALWAYS CALL the tool results using $result_N where N is the index of the tool result in the order they were called, DO NOT PASS BY VALUE.

# Once all steps in the prompt are completed write an end message, stick to the instructions of the user please, dont add any extra information or steps.

# Be careful with decimal points and values. 

# If any of these instructions are not followed, the output will be considered invalid and the system will crash"""

# tool_dict = {
#     "tools": [
#         {
#             "name": "calculator",
#             "description": "Evaluates SCALAR mathematical expressions.",
#             "parameters": {
#                 "expression": {
#                     "description": "The mathematical expression to evaluate.",
#                     "type": "str",
#                     "example": "4**2",
#                     }
#             }
#         },
#         {
#             "name": "get_date",
#             "description": "Returns current date in YYYYMMDD format.",
#             "parameters": {
#                 # No parameters required
#             },
#             "example": {}
#         },
#         {
#             "name": "python_shell",
#             "description": "Executes Python code and returns its result.",
#             "parameters": {
#                 "code": {
#                     "description": "The Python code to execute.",
#                     "type": "str",
#                     "example": "result = 2 + 2"
#                 }
#             }
#         },
#         {
#             "name": "get_weather_details",
#             "description": "Fetches weather details for a location.",
#             "parameters": {
#                 "location": {
#                     "description": "The location for which to retrieve weather information.",
#                     "type": "str",
#                     "example": "New York"
#                 }
#             }
#         }
#     ]
# }


In [ ]:
# system_msg = """
# You are a helpful AI assistant with access to tools. Before acting on a query, list out all your upcoming actions. When you need to use a tool, list what you're gonna do, with what values and then call:
# 1. Output a VALID JSON object with 'name' and 'parameters'. For example:
#    {"name": "calculator", "parameters": {"expression": "25 * 48"}}
#    {"name": "get_date", "parameters": {}}
#    {"name": "python_shell", "parameters": {"code": "result = 2 + 2"}}
#    {"name": "get_weather_details", "parameters": {"location": "New York"}}
# 2. Wait for the tool result, once you get it, say something like "Now that we have the `result_value`, we can proceed with the next step."
# 3. Continue your response using the tool result

# Available tools:
# - calculator: Evaluates mathematical expressions (requires "expression" parameter)
# - get_date: Returns current date in YYYYMMDD format (requires empty {} parameter)
# - python_shell: Executes Python code and returns its result (requires "code" parameter)
# - get_weather_details: Fetches weather details for a location (requires "location" parameter)

# Tool results will be provided in the format: [Tool <name> returned: <result>]

# If you encounter errors, or are asked something which the tools dont support switch to the python_shell tool and always use standard libraries, double-check variable names, return types and print outputs.

# ALWAYS CALL the tool results using $result_N where N is the index of the tool result in the order they were called, DO NOT PASS BY VALUE.

# Once all steps in the prompt are completed write an end message.

# Be careful with decimal points and values. 

# Print a tick mark (✓) at the end of your step to indicate completion.
# """


In [ ]:
# # system_msg = """You are a helpful AI assistant with access to these tools:

# # Before acting on a query, list out all your upcoming actions. When you need to use a tool, list what you're gonna do, with what values and then call:
# # 1. Output a JSON object with 'name' and 'parameters'. For example:
# #     {"name": <tool_name>, "parameters": {}}
# # 2. Wait for the tool result, once you get it, say something like "Now that we have the `result_value`, we can proceed with the next step."
# # 3. Continue your response using the tool result.

# # Tool results will be provided in the format: [Tool <name> returned: <result>]

# # If you encounter errors, or are asked something which the tools don't support, switch to the python_shell tool and always use standard libraries, double-check variable names, return types and print outputs.
# # ALWAYS CALL the tool results using $result_N where N is the index of the tool result in the order they were called, DO NOT PASS BY VALUE.
# # Once all steps in the prompt are completed, write an end message.
# # Be careful with decimal points and values.
# # Print a tick mark (✓) at the end of your step to indicate completion.
# # """

# tool_dict = {
#     "tools": [
#         {
#             "name": "calculator",
#             "description": "Evaluates SCALAR mathematical expressions.",
#             "parameters": {
#                 "expression": {
#                     "description": "The mathematical expression to evaluate.",
#                     "type": "str",
#                     "example": "4**2",
#                     "required": True,
#                     "available_func": [
#                         "sin", "cos", "tan", "log", "exp", "sqrt", "floor", "ceil",
#                         "asin", "acos", "atan", "degrees", "radians", "pi", "e", "pow"
#                     ]
#                 }
#             }
#         },
#         {
#             "name": "get_date",
#             "description": "Returns current date in YYYYMMDD format.",
#             "parameters": {}  # no parameters needed
#         },
#         {
#             "name": "python_shell",
#             "description": "Executes Python code and returns its result.",
#             "parameters": {
#                 "code": {
#                     "description": "The Python code to execute.",
#                     "type": "str",
#                     "example": "print(2 + 2)",
#                     "required": True
#                 }
#             }
#         },
#         {
#             "name": "get_weather_details",
#             "description": "Fetches weather details for a location.",
#             "parameters": {
#                 "location": {
#                     "description": "The location to retrieve weather information for.",
#                     "type": "str",
#                     "example": "New York",
#                     "required": True
#                 }
#             }
#         }
#     ]
# }